# Geog 80 Final - Ryan Meyer
## COVID-19 Pandemic in the United States

### Section 1: Data loading and processing
First, we shall load in the covid case data, US Census 2019 population estimates, as well as some GeoJSON data containing county geometry. Some cleaning needs to be done to match the format of FIPS codes between datasets. Additionally, we will clean the population dataset and join it via census area names. Finally, we divide the joined table by the data from the US population table to yield the final dataset: a dataframe of covid cases per capita for each county for each day.

In [78]:
import pandas as pd
import json
from urllib.request import urlopen
import warnings

caseDataFile = "https://raw.githubusercontent.com/ryanpmeyer/Geog80Final/master/data/time_series_covid19_confirmed_US.csv"
populationDataFile = "https://raw.githubusercontent.com/ryanpmeyer/Geog80Final/master/data/co-est2019-alldata.csv"
geoJSONFile = 'https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json'
eventsFile = 'https://raw.githubusercontent.com/ryanpmeyer/Geog80Final/master/data/EVENTS.csv'

def getData():
  with urlopen(geoJSONFile) as response:
    counties = json.load(response)

  eventsData = pd.read_csv(eventsFile)

  # get covid case data, clean FIPS code
  covid_cases_raw = pd.read_csv(caseDataFile)
  covid_cases = covid_cases_raw.dropna().reindex()
  old_fips = covid_cases['FIPS'].astype('int32').astype(str)
  max_fips_code_len = max([len(x) for x in old_fips])
  new_fips = list(old_fips)
  for i in range(len(old_fips)):
      while len(new_fips[i]) < max_fips_code_len:
          new_fips[i] = '0' + new_fips[i]
  covid_cases['FIPS'] = new_fips

  # get US population data, clean county names
  us_pop_raw = pd.read_csv(populationDataFile, encoding='latin1')
  us_pop_raw['CTY_OG'] = us_pop_raw['CTYNAME']
  us_pop = us_pop_raw[us_pop_raw['CTYNAME'] != us_pop_raw['STNAME']]
  strings2remove = [' County',' Borough',' Census Area',' Municipality',' Parish', ' city', ' City']
  for string in strings2remove:
      us_pop['CTYNAME'] = us_pop['CTYNAME'].str.replace(string,"",case=False)
  us_pop['CTYNAME'] = us_pop['CTYNAME'].str.strip()

  # remove data from population with duplicate locations
  indicies_to_remove = []
  for state in set(us_pop['STNAME']):
    counties_in_state = us_pop[us_pop['STNAME']==state]
    for cty in set(counties_in_state['CTYNAME']):
      rows = counties_in_state[counties_in_state['CTYNAME']==cty]
      if len(rows) > 1:
        rows_index = rows.index
        highest_pop_i = rows['POPESTIMATE2019'].argmax()
        for i in rows_index:
          if i != highest_pop_i:
            indicies_to_remove.append(i)
  us_pop.drop(index=indicies_to_remove,inplace=True)
  us_pop.reindex(copy=False)

  # join tables and divide case nums by population
  us_pop['place'] = us_pop['CTYNAME'] + ',' + us_pop['STNAME']
  covid_cases['place'] = covid_cases['Admin2'] + ',' + covid_cases['Province_State']
  joined = covid_cases.set_index('place').join(us_pop.set_index("place")[['POPESTIMATE2019','CTY_OG']],
              how='inner', lsuffix='County', rsuffix='').sort_index()

  # set county names back to original
  joined['Area_Name'] = joined['CTY_OG'] + ', '+joined['Province_State']
    
  # get case data per 10000 people
  for col in joined.columns[11:len(joined.columns)-2]:
      joined[col] = joined[col] / joined['POPESTIMATE2019'] * 10000 # per 10000 people



  used_fips = set(joined['FIPS'])
  # remove data from GeoJSON without matching FIPS code in case data
  
  cleaned_features = list(counties['features'])
  for feature in counties['features']:
    if feature['id'] not in used_fips:
      cleaned_features.remove(feature)
  counties['features'] = cleaned_features

  return counties, joined, eventsData

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    counties, caseData, eventsData = getData()
display(caseData.head(5))
display(eventsData.head(5))

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,POPESTIMATE2019,CTY_OG,Area_Name
place,,,,,,,,,,,,,,,,,,,,,
"Abbeville,South Carolina",84045001,US,USA,840,45001,Abbeville,South Carolina,US,34.223334,-82.461707,...,380.804827,386.512823,391.405390,391.813104,394.259388,396.297957,398.744241,10000.0,Abbeville County,"Abbeville County, South Carolina"
"Acadia,Louisiana",84022001,US,USA,840,22001,Acadia,Louisiana,US,30.295065,-92.414197,...,625.997260,636.634701,636.634701,641.792248,641.147554,649.367395,657.426062,10000.0,Acadia Parish,"Acadia Parish, Louisiana"
"Accomack,Virginia",84051001,US,USA,840,51001,Accomack,Virginia,US,37.767072,-75.632346,...,409.704171,411.560837,414.655279,416.202500,416.511945,419.915831,420.534720,10000.0,Accomack County,"Accomack County, Virginia"
"Ada,Idaho",84016001,US,USA,840,16001,Ada,Idaho,US,43.452658,-116.241552,...,505.204667,515.400125,521.837176,526.675346,533.673043,542.809503,550.762375,10000.0,Ada County,"Ada County, Idaho"
"Adair,Iowa",84019001,US,USA,840,19001,Adair,Iowa,US,41.330756,-94.471059,...,599.832215,599.832215,615.212528,616.610738,616.610738,620.805369,626.398210,10000.0,Adair County,"Adair County, Iowa"


,DATE,LAT,LON,HEADLINE,DESCRIPTION,LINK
0,1/21/20,47.442224,-122.302510,First Travel-related Case of 2019 Novel Corona...,This first instance of a 2019-nCoV infected re...,https://www.cdc.gov/media/releases/2020/p0121-...
1,1/31/20,0.000000,0.000000,WHO Issues Global Health Emergency,As case numbers quickly began to jump exponent...,https://www.ajmc.com/view/what-were-reading-gl...
2,2/3/20,0.000000,0.000000,US Declares Public Health Emergency,"3 days after the WHO, the Trump administration...",https://www.jurist.org/news/2020/02/trump-admi...
3,2/25/20,0.000000,0.000000,CDC Says COVID-19 Is Heading toward Pandemic S...,Pandemics have 3 main factors: illness resulti...,https://www.cdc.gov/media/releases/2020/t0225-...
4,2/26,42.359896,-71.051103,Biogen meeting leads to COVID-19 spread in mul...,After a February meeting with senior executive...,https://www.nytimes.com/2020/12/11/us/biogen-c...


### Section 2: Choropleth analysis

Now we have data for each US county's confirmed cases per capita for each day from 1/22 - 12/2. Next we will look at a choropleth map of this data.

In [71]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipyleaflet
import ipywidgets as widgets
import datetime
import branca
import numpy as np

NUMDAYS = 315
STARTDATE = datetime.date(2020,1,22)
DATES = [STARTDATE + datetime.timedelta(days=x) for x in range(NUMDAYS)]
max_caseload = max(caseData['12/2/20'])



def get_colorscale(max_val=max_caseload):
  return branca.colormap.linear.BuPu_09.scale(0, max_val)

def get_date_string(date):
  date_string = date.strftime("%x") # format date into D/M/Y
  if date_string[0] == '0': # no leading zeros in month
    date_string = date_string[1:]
  if date_string[date_string.find('/')+1] == '0': # no leading zeros in day
    date_string = date_string[0:date_string.find('/')+1] + date_string[date_string.find('/')+2:]
  return date_string

def get_styler(data_series, absolute=False):
    
    def style(feature):
      max_val = max_caseload if absolute else max(data_series)
      colorscale = get_colorscale(max_val)
      value = data_series.get(feature['id'])
      if value is None:
        value = -1
      return {
          'color': 'black',
          'fillColor': colorscale(value) if value != -1 else 'black'
      }
    return style

def get_legend(data_series, absolute=False):
    max_val = max_caseload if absolute else max(data_series)
    colorscale = get_colorscale(max_val)
    legend_dict = {str(round(val,4)):colorscale(val) for val in np.linspace(0,max_val,6)}
    return legend_dict

def get_choropleth(date, absolute=False):
  old_date = date
  date_string = get_date_string(date)
  data_series = caseData.set_index('FIPS')[date_string]

  m = ipyleaflet.Map(center=[48,-102],zoom=4)

  choropleth_layer = ipyleaflet.GeoJSON(
      data=counties,
      style={
          'opacity': 1.,
          'fillOpacity': 0.5,
          'weight': .3
      },
      hover_style={
          'fillOpacity': 1.,
          'weight': .6
      },
      style_callback=get_styler(data_series)
  )
  m.add_layer(choropleth_layer)

  # UI changes upon hover action
  def hover_handler(event=None, feature=None, id=None, properties=None):
        case_val = round(data_series[id], 4)
        hover_popup.value = '<b>'+caseData.set_index('FIPS')['Area_Name'][id]+'</b>: '+str(case_val)
  hover_popup = widgets.HTML()
  hover_popup.layout.margin = '0px 20px 20px 20px'
  m.add_control(ipyleaflet.WidgetControl(widget=hover_popup,position='bottomleft'))
  choropleth_layer.on_hover(hover_handler)
    
  # Markers for event information ("essay" part)
  event_markers = []
  for event_i in events.index:
    
    
  

  date_picker = widgets.DatePicker(value=STARTDATE, enabled_dates=DATES, description='Date: ')
  date_control = ipyleaflet.WidgetControl(widget=date_picker,position='topright')
  m.add_control(date_control)
  legend_control = ipyleaflet.LegendControl(get_legend(data_series, absolute),
                                            name = 'cases/10k residents',
                                            position = 'bottomright', 
  )
  m.add_control(legend_control)
    
  def update(date):
    global old_date
    if date not in DATES:
        date_picker.value = old_date
        return;
    old_date = date
    date_string = get_date_string(date)
    hover_popup.value = "Updating map to: "+ date_string
    data_series = caseData.set_index('FIPS')[date_string]
    legend_control.legends = get_legend(data_series, absolute) # update legend
    choropleth_layer.style_callback = get_styler(data_series, absolute) # update style (colors)
    hover_popup.value = "Map updated."

  return m, update, date_picker



def get_interactive_map(start_date=STARTDATE):
  m, update, date_picker = get_choropleth(start_date)
  interact(update, date=date_picker)
  return m

get_interactive_map()

interactive(children=(DatePicker(value=datetime.date(2020, 1, 22), description='Date: '), Output()), _dom_clas…

Map(center=[48, -102], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…